In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

sup_min = 10
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

prop = propiedades.loc[(propiedades.Year >= 2016), :]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [3]:
set_entrenamiento = prop.loc[(prop.Year >= 2016) &((prop.Year < 2017) | (prop.Month < 6))\
                                    ,columnas_precio]
set_pruebas = prop.loc[(prop.Year == 2017) & (prop.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [4]:
def predecir(x, predictor, clusters):
    indice = cluster.predict([x])[0]
    return predictor[indice].predict([x])[0]

# Obtenemos la cantidad de clusters

In [8]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 1.0

lista = [2, 10,20, 30, 40, 50, 100, 200, 300]


minimo = {'minimo': (0, np.inf)}

for l in lista:
    cluster = KMeans(n_clusters = l)
    m = cluster.fit_predict(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_entrenamiento.loc[:, 'cluster'] = m

    predictores = {}
    for i in range (l):
        tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth,\
                                               min_samples_split = min_sample, min_samples_leaf = min_leaf,\
                                               max_features = feature)
        entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
        datos = entrenamiento.loc[:, columnas]
        resultado = entrenamiento.loc[:,'price_aprox_usd']
        tree.fit(datos, resultado)
        predictores[i] = tree
    
    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

    error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
    
    if error < minimo['minimo'][1]:
        minimo['minimo'] = (l, error)
        
c, error = minimo['minimo']
print("Cantidad de clusters = {}, error = {}, cant_digitos = {}".format(c, error, len(str(int(round(error,0))))))

Cantidad de clusters = 50, error = 436687242.177, cant_digitos = 9


In [97]:
cantidad_clusters = 50

cluster = KMeans(n_clusters = cantidad_clusters)
m = cluster.fit_predict(set_entrenamiento_datos,set_entrenamiento_resultado)

In [98]:
set_entrenamiento.loc[:, 'cluster'] = m

# Con decision Tree

In [7]:
from sklearn.tree import DecisionTreeRegressor

In [38]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 1.0

predictores = {}
for i in range (cantidad_clusters):
    tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth,\
                                           min_samples_split = min_sample, min_samples_leaf = min_leaf,\
                                           max_features = feature)
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    tree.fit(datos, resultado)
    predictores[i] = tree

In [39]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 476743287.157, cant_digitos = 9


# Con Bagging Regressor

In [40]:
from sklearn.ensemble import BaggingRegressor

In [41]:
estimator = 5
sample = 1.0
feature = 1.0

predictores = {}
for i in range (cantidad_clusters):
    bag = BaggingRegressor(n_estimators = estimator, max_samples = sample, max_features = feature)    
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    bag.fit(datos, resultado)
    predictores[i] = bag

In [42]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 574063832.502, cant_digitos = 9


# Con KNN

In [43]:
from sklearn.neighbors import KNeighborsRegressor

In [56]:
k = 1
d = 1

predictores = {}
for i in range (cantidad_clusters):
    knn = KNeighborsRegressor(n_neighbors = k, p = d)    
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    knn.fit(datos, resultado)
    predictores[i] = knn

In [57]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 826885555.257, cant_digitos = 9


# Con Lightgbm

In [52]:
import lightgbm as lgb

In [59]:
bt = 'goss'
lr = 1.5
n = 10000

predictores = {}
for i in range (cantidad_clusters):
    lgbm = lgb.LGBMRegressor(n_estimators = n, learning_rate = lr, boosting_type = bt)
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    lgbm.fit(datos, resultado)
    predictores[i] = lgbm

In [60]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 568903278.789, cant_digitos = 9


# Con Extra Trees

In [58]:
from sklearn.ensemble import ExtraTreesRegressor

In [59]:
criterio = 'mse'
max_depth = None
n_estimators = 50
max_features = 1.0

predictores = {}
for i in range (cantidad_clusters):
    tree = ExtraTreesRegressor(n_estimators = 100)
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    tree.fit(datos, resultado)
    predictores[i] = tree

In [60]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 338214428.949, cant_digitos = 9


In [61]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")
analizar.loc[:,'resultado'] = analizar.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

In [62]:
analizar.resultado.describe()

count     14166.000000
mean     182706.856416
std      117332.423253
min       22476.000000
25%      105353.201350
50%      150971.035269
75%      220302.000000
max      924463.333333
Name: resultado, dtype: float64

In [63]:
resultado = analizar.loc[:,['id','price_usd']]
resultado.to_csv('resultados/Extra_Trees_con_Kmeans.csv', index = False)

# Con Gradient Boosting

In [64]:
from sklearn.ensemble import GradientBoostingRegressor

In [65]:
l = 'ls'
n = 100
d = 20
lr = 0.3

predictores = {}
for i in range (cantidad_clusters):
    gradient = GradientBoostingRegressor(loss = l, n_estimators = n, max_depth = d, learning_rate = lr)
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    gradient.fit(datos, resultado)
    predictores[i] = gradient

In [66]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 487882468.288, cant_digitos = 9


In [67]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")
analizar.loc[:,'resultado'] = analizar.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

In [68]:
analizar.resultado.describe()

count     14166.000000
mean     184264.758655
std      136038.787966
min        5005.030538
25%       96924.014517
50%      144230.617957
75%      223213.073703
max      999997.678445
Name: resultado, dtype: float64

In [69]:
resultado = analizar.loc[:,['id','price_usd']]
resultado.to_csv('resultados/Gradient_Boosting_con_Kmeans.csv', index = False)

# Con Ada Boost

In [70]:
from sklearn.ensemble import AdaBoostRegressor

In [71]:
l = 'square'
lr = 0.3
n = 100

predictores = {}
for i in range (cantidad_clusters):
    ada = AdaBoostRegressor(loss = l, learning_rate = lr, n_estimators = n)
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    ada.fit(datos, resultado)
    predictores[i] = ada

In [72]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 6490206014.6, cant_digitos = 10


In [73]:
l = 'linear'
lr = 0.3
n = 100

predictores = {}
for i in range (cantidad_clusters):
    ada = AdaBoostRegressor(loss = l, learning_rate = lr, n_estimators = n)

    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    ada.fit(datos, resultado)
    predictores[i] = ada

In [74]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 4878089438.17, cant_digitos = 10


# Con XGBoost

In [75]:
import xgboost

In [76]:
d = 10
n = 500
lr = 0.5 
b = 'gbtree'

predictores = {}
for i in range (cantidad_clusters):
    xgb = xgboost.XGBRegressor(n_estimators = n, learning_rate = lr, booster = b, max_depth = d)
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    xgb.fit(datos, resultado)
    predictores[i] = xgb

In [77]:
set_pruebas.loc[:,'resultado'] = xgb.predict(set_pruebas.loc[:,columnas])

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 12799245350.4, cant_digitos = 11


# Con Perceptron

In [78]:
from sklearn.neural_network import MLPRegressor

In [79]:
solver = 'lbfgs'
activation = 'logistic'
tol=1e-10
warm_start = True

predictores = {}
for i in range (cantidad_clusters):
    perceptron = MLPRegressor(solver = solver, activation = activation, tol = tol,  warm_start = warm_start)
    
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    #for k in range(6):
    #    perceptron.fit(datos,resultado)
    perceptron.fit(datos, resultado)
    predictores[i] = perceptron

In [80]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(error,0))))))

error = 9333433950.44, cant_digitos = 10


In [ ]:
# probar de reemplazar esto

In [ ]:
def crear_cluster(cantidad_cluster, data_frame, datos, resultado):
    cluster = KMeans(n_clusters = cantidad_cluster)
    m = cluster.fit_predict(datos, resultado)
    data_frame.loc[:, 'cluster'] = m
    return cluster